In [5]:
import os
import csv
import platform
import pandas as pd
import networkx as nx
from graph_partitioning import GraphPartitioning, utils

run_metrics = True

cols = ["WASTE", "CUT RATIO", "EDGES CUT", "TOTAL COMM VOLUME", "Qds", "CONDUCTANCE", "MAXPERM", "RBSE", "NMI", "FSCORE", "FSCORE RELABEL IMPROVEMENT", "LONELINESS"]

pwd = %pwd

config = {

    "DATA_FILENAME": os.path.join(pwd, "data", "predition_model_tests", "network", "network_$$.txt"),
    "OUTPUT_DIRECTORY": os.path.join(pwd, "output"),

    # Set which algorithm is run for the PREDICTION MODEL.
    # Either: 'FENNEL' or 'SCOTCH'
    "PREDICTION_MODEL_ALGORITHM": "FENNEL",

    # Alternativly, read input file for prediction model.
    # Set to empty to generate prediction model using algorithm value above.
    "PREDICTION_MODEL": "",

    
    "PARTITIONER_ALGORITHM": "FENNEL",

    # File containing simulated arrivals. This is used in simulating nodes
    # arriving at the shelter. Nodes represented by line number; value of
    # 1 represents a node as arrived; value of 0 represents the node as not
    # arrived or needing a shelter.
    "SIMULATED_ARRIVAL_FILE": os.path.join(pwd,
                                           "data",
                                           "predition_model_tests",
                                           "dataset_6_randomize",
                                           "simulated_arrival_list",
                                           "percentage_of_prediction_randomized_££",
                                           "arrival_££_$$.txt"
                                          ),

    # File containing the prediction of a node arriving. This is different to the
    # simulated arrivals, the values in this file are known before the disaster.
    "PREDICTION_LIST_FILE": os.path.join(pwd,
                                         "data",
                                         "predition_model_tests",
                                         "dataset_6_randomize",
                                         "prediction_list",
                                         "prediction_$$.txt"
                                        ),

    # File containing the geographic location of each node, in "x,y" format.
    "POPULATION_LOCATION_FILE": os.path.join(pwd,
                                             "data",
                                             "predition_model_tests",
                                             "coordinates",
                                             "coordinates_$$.txt"
                                            ),

    # Number of shelters
    "num_partitions": 4,

    # The number of iterations when making prediction model
    "num_iterations": 1,

    # Percentage of prediction model to use before discarding
    # When set to 0, prediction model is discarded, useful for one-shot
    "prediction_model_cut_off": .0,

    # Alpha value used in one-shot (when restream_batches set to 1)
    "one_shot_alpha": 0.5,
    
    "use_one_shot_alpha" : False,
    
    # Number of arrivals to batch before recalculating alpha and restreaming.
    "restream_batches": 50,

    # When the batch size is reached: if set to True, each node is assigned
    # individually as first in first out. If set to False, the entire batch
    # is processed and empty before working on the next batch.
    "sliding_window": False,

    # Create virtual nodes based on prediction model
    "use_virtual_nodes": False,

    # Virtual nodes: edge weight
    "virtual_edge_weight": 1.0,
    
    # Loneliness score parameter. Used when scoring a partition by how many
    # lonely nodes exist.
    "loneliness_score_param": 1.2,


    ####
    # GRAPH MODIFICATION FUNCTIONS

    # Also enables the edge calculation function.
    "graph_modification_functions": True,

    # If set, the node weight is set to 100 if the node arrives at the shelter,
    # otherwise the node is removed from the graph.
    "alter_arrived_node_weight_to_100": False,

    # Uses generalized additive models from R to generate prediction of nodes not
    # arrived. This sets the node weight on unarrived nodes the the prediction
    # given by a GAM.
    # Needs POPULATION_LOCATION_FILE to be set.
    "alter_node_weight_to_gam_prediction": False,
    
    # Enables edge expansion when graph_modification_functions is set to true
    "edge_expansion_enabled": True,

    # The value of 'k' used in the GAM will be the number of nodes arrived until
    # it reaches this max value.
    "gam_k_value": 100,

    # Alter the edge weight for nodes that haven't arrived. This is a way to
    # de-emphasise the prediction model for the unknown nodes.
    "prediction_model_emphasis": 1.0,
    
    "compute_metrics_enabled" : True,
    
    # This applies the prediction_list_file node weights onto the nodes in the graph
    # when the prediction model is being computed and then removes the weights
    # for the cutoff and batch arrival modes
    "apply_prediction_model_weights": True,

    "SCOTCH_LIB_PATH": os.path.join(pwd, "libs/scotch/macOS/libscotch.dylib")
    if 'Darwin' in platform.system()
    else "/usr/local/lib/libscotch.so",
    
    # Path to the PaToH shared library
    "PATOH_LIB_PATH": os.path.join(pwd, "libs/patoh/lib/macOS/libpatoh.dylib")
    if 'Darwin' in platform.system()
    else os.path.join(pwd, "libs/patoh/lib/linux/libpatoh.so"),
    
    "PATOH_ITERATIONS": 5,
        
    # Expansion modes: 'avg_node_weight', 'total_node_weight', 'smallest_node_weight'
    # 'largest_node_weight'
    # add '_squared' or '_sqrt' at the end of any of the above for ^2 or sqrt(weight)
    # i.e. 'avg_node_weight_squared
    "PATOH_HYPEREDGE_EXPANSION_MODE": 'no_expansion',
    
    # Edge Expansion: average, total, minimum, maximum, product, product_squared, sqrt_product
    "EDGE_EXPANSION_MODE" : 'total',
    
    # Whether nodes should be reordered using a centrality metric for optimal node assignments in batch mode
    # This is specific to FENNEL and at the moment Leverage Centrality is used to compute new noder orders
    "FENNEL_NODE_REORDERING_ENABLED": False,
    
    # Whether the Friend of a Friend scoring system is active during FENNEL partitioning.
    # FOAF employs information about a node's friends to determine the best partition when
    # this node arrives at a shelter and no shelter has friends already arrived
    "FENNEL_FRIEND_OF_A_FRIEND_ENABLED": False,
    
    # Alters how much information to print. Keep it at 1 for this notebook.
    # 0 - will print nothing, useful for batch operations.
    # 1 - prints basic information on assignments and operations.
    # 2 - prints more information as it batches arrivals.
    "verbose": 1
}

gp = GraphPartitioning(config)

# Optional: shuffle the order of nodes arriving
# Arrival order should not be shuffled if using GAM to alter node weights
#random.shuffle(gp.arrival_order)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
import scipy
from copy import deepcopy
import time

iterations = 60

pm_cutoff = []
for i in range(0, 21):
    pm_cutoff.append(i * 0.05)

#virtual_edge_weight = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]

for mode in pm_cutoff:

    metricsDataPrediction = []
    metricsDataAssign = []
    
    dataQdsOv = []
    dataCondOv = []

    # batches of 50 - use 10 restreaming iterations on FENNEL
    config['num_iterations'] = 10
        
    config['prediction_model_cut_off'] = mode
    #config['alter_node_weight_to_gam_prediction'] = False
    #config["use_virtual_nodes"] = True

    # Virtual nodes: edge weight
    #config["virtual_edge_weight"] = mode


    print('Mode', mode)
    elapsed_times = []
    for i in range(0, iterations):
        # how many networks
        if (i % 20) == 0:
            print('Mode', mode, 'Iteration', str(i))
        
        conf = deepcopy(config)

        correctedness = 60
        
        conf["DATA_FILENAME"] = conf["DATA_FILENAME"].replace('$$', str(i + 1))
        
        conf["SIMULATED_ARRIVAL_FILE"] = conf["SIMULATED_ARRIVAL_FILE"].replace('$$', str(i + 1))
        conf["SIMULATED_ARRIVAL_FILE"] = conf["SIMULATED_ARRIVAL_FILE"].replace('££', str(correctedness))

        conf["PREDICTION_LIST_FILE"] = conf["PREDICTION_LIST_FILE"].replace('$$', str(i + 1))
        conf["POPULATION_LOCATION_FILE"] = conf["POPULATION_LOCATION_FILE"].replace('$$', str(i + 1))


        with GraphPartitioning(conf) as gp:
            gp.verbose = 0
            
            start_time = time.time()
            
            gp.load_network()
            gp.init_partitioner()

            m = gp.prediction_model()
            m = gp.assign_cut_off()
            m = gp.batch_arrival()

            elapsed_time = time.time() - start_time
            elapsed_times.append(elapsed_time)
            
            totalM = len(m)
            metricsDataPrediction.append(m[totalM - 1])
    
    print('Average Elapsed Time =', scipy.mean(elapsed_times))

    waste = ''
    cutratio = ''
    ec = ''
    tcv = ''
    qds = ''
    conductance = ''
    maxperm = ''
    rbse = ''
    nmi = ''
    lonliness = ''
    fscore = ''
    fscoreimprove = ''
        
    qdsOv = ''
    condOv = ''

    dataWaste = []
    dataCutRatio = []
    dataEC = []
    dataTCV = [] 
    dataQDS = []
    dataCOND = []
    dataMAXPERM = []
    dataRBSE = []
    dataNMI = []
    dataLonliness = []
    dataFscore = []
    dataFscoreImprove = []

    
    for i in range(0, iterations):
        dataWaste.append(metricsDataPrediction[i][0])        
        dataCutRatio.append(metricsDataPrediction[i][1])
        dataEC.append(metricsDataPrediction[i][2])
        dataTCV.append(metricsDataPrediction[i][3])
        dataQDS.append(metricsDataPrediction[i][4])
        dataCOND.append(metricsDataPrediction[i][5])
        dataMAXPERM.append(metricsDataPrediction[i][6])
        dataRBSE.append(metricsDataPrediction[i][7])
        dataNMI.append(metricsDataPrediction[i][8])        
        dataFscore.append(metricsDataPrediction[i][9])        
        dataFscoreImprove.append(metricsDataPrediction[i][10])        
        dataLonliness.append(metricsDataPrediction[i][11])


        if(len(waste)):
            waste = waste + ','
        waste = waste + str(metricsDataPrediction[i][0])

        if(len(cutratio)):
            cutratio = cutratio + ','
        cutratio = cutratio + str(metricsDataPrediction[i][1])

        if(len(ec)):
            ec = ec + ','
        ec = ec + str(metricsDataPrediction[i][2])
        
        if(len(tcv)):
            tcv = tcv + ','
        tcv = tcv + str(metricsDataPrediction[i][3])

        if(len(qds)):
            qds = qds + ','
        qds = qds + str(metricsDataPrediction[i][4])

        if(len(conductance)):
            conductance = conductance + ','
        conductance = conductance + str(metricsDataPrediction[i][5])

        if(len(maxperm)):
            maxperm = maxperm + ','
        maxperm = maxperm + str(metricsDataPrediction[i][6])

        if(len(rbse)):
            rbse = rbse + ','
        rbse = rbse + str(metricsDataPrediction[i][7])


        
        if(len(nmi)):
            nmi = nmi + ','
        nmi = nmi + str(metricsDataPrediction[i][8])

        if(len(fscore)):
            fscore = fscore + ','
        fscore = fscore + str(metricsDataPrediction[i][9])

        if(len(fscoreimprove)):
            fscoreimprove = fscoreimprove + ','
        fscoreimprove = fscoreimprove + str(metricsDataPrediction[i][10])
        
        if(len(lonliness)):
            lonliness = lonliness + ','
        lonliness = lonliness + str(dataLonliness[i])
        
    '''
    waste = 'WASTE,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataWaste)) + ',' + str(scipy.std(dataWaste)) + ',' + waste

    cutratio = 'CUT_RATIO,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataCutRatio)) + ',' + str(scipy.std(dataCutRatio)) + ',' + cutratio
    ec = 'EC,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataEC)) + ',' + str(scipy.std(dataEC)) + ',' + ec
    tcv = 'TCV,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataTCV)) + ',' + str(scipy.std(dataTCV)) + ',' + tcv

    lonliness = "LONELINESS," + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataLonliness)) + ',' + str(scipy.std(dataLonliness)) + ',' + lonliness

    qds = 'QDS,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataQDS)) + ',' + str(scipy.std(dataQDS)) + ',' + qds
    conductance = 'CONDUCTANCE,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataCOND)) + ',' + str(scipy.std(dataCOND)) + ',' + conductance
    maxperm = 'MAXPERM,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataMAXPERM)) + ',' + str(scipy.std(dataMAXPERM)) + ',' + maxperm
    rbse = 'RBSE,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataRBSE)) + ',' + str(scipy.std(dataRBSE)) + ',' + rbse

    nmi = 'NMI,' + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataNMI)) + ',' + str(scipy.std(dataNMI)) + ',' + nmi

    fscore = "FSCORE," + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataFscore)) + ',' + str(scipy.std(dataFscore)) + ',' + fscore
    fscoreimprove = "FSCORE_IMPROVE," + 'virtual_e_w_' + str(int(10 * config['virtual_edge_weight'])) + ',' + str(scipy.mean(dataFscoreImprove)) + ',' + str(scipy.std(dataFscoreImprove)) + ',' + fscoreimprove
    
    '''
    waste = 'WASTE,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataWaste)) + ',' + str(scipy.std(dataWaste)) + ',' + waste

    cutratio = 'CUT_RATIO,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataCutRatio)) + ',' + str(scipy.std(dataCutRatio)) + ',' + cutratio
    ec = 'EC,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataEC)) + ',' + str(scipy.std(dataEC)) + ',' + ec
    tcv = 'TCV,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataTCV)) + ',' + str(scipy.std(dataTCV)) + ',' + tcv

    lonliness = "LONELINESS," + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataLonliness)) + ',' + str(scipy.std(dataLonliness)) + ',' + lonliness

    qds = 'QDS,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataQDS)) + ',' + str(scipy.std(dataQDS)) + ',' + qds
    conductance = 'CONDUCTANCE,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataCOND)) + ',' + str(scipy.std(dataCOND)) + ',' + conductance
    maxperm = 'MAXPERM,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataMAXPERM)) + ',' + str(scipy.std(dataMAXPERM)) + ',' + maxperm
    rbse = 'RBSE,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataRBSE)) + ',' + str(scipy.std(dataRBSE)) + ',' + rbse

    nmi = 'NMI,' + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataNMI)) + ',' + str(scipy.std(dataNMI)) + ',' + nmi

    fscore = "FSCORE," + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataFscore)) + ',' + str(scipy.std(dataFscore)) + ',' + fscore
    fscoreimprove = "FSCORE_IMPROVE," + 'pm_cutoff_' + str(int(100 * config['prediction_model_cut_off'])) + ',' + str(scipy.mean(dataFscoreImprove)) + ',' + str(scipy.std(dataFscoreImprove)) + ',' + fscoreimprove
    ''''''

    print(cutratio)
    print(ec)
    print(tcv)
    print(lonliness)
    print(qds)
    print(conductance)
    print(maxperm)
    print(rbse)
    print(nmi)
    print(fscore)
    print(fscoreimprove)

Mode 0.0
Mode 0.0 Iteration 0
Mode 0.0 Iteration 20
Mode 0.0 Iteration 40
Average Elapsed Time = 239.204368711
CUT_RATIO,pm_cutoff_0,0.179514044837,0.0190526687988,0.18648018648,0.21420518602,0.218225419664,0.19900990099,0.163355408389,0.204152249135,0.177282377919,0.192008879023,0.178240740741,0.148664343786,0.168408826945,0.197263397948,0.171129220023,0.191402251791,0.134328358209,0.18931475029,0.171021377672,0.179951690821,0.167550371156,0.169076751947,0.150549450549,0.154171066526,0.184337349398,0.204353083434,0.19305019305,0.174468085106,0.173572228443,0.183313748531,0.158653846154,0.202964652223,0.188335358445,0.181324647123,0.167410714286,0.200661521499,0.135552913199,0.163104611924,0.168574401665,0.186073059361,0.182126696833,0.193965517241,0.168058455115,0.15578465063,0.181922196796,0.186403508772,0.18715393134,0.157480314961,0.182831661093,0.198609731877,0.163265306122,0.143340857788,0.196571428571,0.196332254585,0.178128523112,0.175182481752,0.168564920273,0.205811138015,0.2